In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
import pandas as pd
df = pd.read_excel('/content/drive/Shareddrives/실전기계_project/data_labeling/data_labeling_3.xlsx', sheet_name=None)
df = pd.concat(df, ignore_index=True)
df = df[['document', 'spo_label']]

In [43]:
df.head()

,document,spo_label
0,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,0.0
1,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",0.0
2,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,0.0
3,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,0.0
4,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,0.0


In [44]:
df.shape

(5251, 2)

In [45]:
df['spo_label'].value_counts()

1.0    627
0.0    155
Name: spo_label, dtype: int64

In [46]:
# column명 변경
df.rename(columns={'document':'content', 'spo_label':'label'}, inplace=True)

In [47]:
df = df.fillna(0)

In [48]:
df['label'].value_counts()

0.0    4624
1.0     627
Name: label, dtype: int64

In [49]:
df_0 = df[df['label'] == 0]
df_1 = df[df['label'] == 1]

In [50]:
df_0 = df_0[:627]

In [51]:
df_0_test = df_0[:110]
df_1_test = df_1[:110]
df_0_train = df_0[110:]
df_1_train = df_1[110:]

In [52]:
test = pd.concat([df_0_test, df_1_test]) # 426->627
test = test.sample(frac=1)

train = pd.concat([df_0_train, df_1_train]) # 426->627
train = train.sample(frac=1)

In [53]:
train['label'].value_counts()

0.0    517
1.0    517
Name: label, dtype: int64

In [54]:
test['label'].value_counts()

0.0    110
1.0    110
Name: label, dtype: int64

In [26]:
test['label'].value_counts()

1.0    119
0.0    101
Name: label, dtype: int64

In [58]:
test.to_csv("/content/drive/Shareddrives/실전기계_project/data/data_spo_test_1210.csv")

In [59]:
train.to_csv("/content/drive/Shareddrives/실전기계_project/data/data_spo_aug_X_train_1210.csv")

In [60]:
import random
import pickle
import re

wordnet = {}
with open("/content/drive/Shareddrives/실전기계_project/wordnet.pickle", "rb") as f:
	wordnet = pickle.load(f)
 
 

In [61]:
# 한글만 남기고 나머지는 삭제
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

	return parseText



########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break

	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")

	else:
		new_words = ""

	return new_words


def get_synonyms(word):
	synomyms = []

	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass

	return synomyms

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	
	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""

		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)


In [110]:
def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=5):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1

	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))

	
	
	
	
	
	
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	



	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)

	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	augmented_sentences.append(sentence)

	return augmented_sentences

In [ ]:
 

  
  
  
  
  
  # sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
  
  # rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
  
  # rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
  
  

	

IndentationError: ignored

In [111]:
df_spo_1 = train[train['label']==1]
df_spo_0 = train[train['label']==0]

In [112]:
df_spo_1_aug = df_spo_1['content'].apply(EDA)
df_spo_0_aug = df_spo_0['content'].apply(EDA)

In [113]:
aug_1 = []
for i in range(len(df_spo_1_aug)):
  for j in range(len(df_spo_1_aug.iloc[i])):
    aug_1.append(df_spo_1_aug.iloc[i][j])

In [114]:
aug_0 = []
for i in range(len(df_spo_0_aug)):
  for j in range(len(df_spo_0_aug.iloc[i])):
    aug_0.append(df_spo_0_aug.iloc[i][j])

In [115]:
df_augmentation_1 = pd.DataFrame({'content': aug_1,
             'label': 1})
df_augmentation_0 = pd.DataFrame({'content': aug_0,
             'label': 0})

In [116]:
df_augmentation_1[:6]

,content,label
0,전력삼국지 몇번다시보다가 다시 영화봤는데 이해가 빠르네요 관우랑 안량이랑 일기토할때...,1
1,전력삼국지 몇번다시보다가 다시 영화봤는데 이해가 빠르네요 관우랑 안량이랑 일기토할때...,1
2,전력삼국지 몇번다시보다가 다시 영화봤는데 이해가 빠르네요 관우랑 안량이랑 일기토할때...,1
3,"보고싶은 것만 보고, 본것을 믿으며 믿는만큼 생각하며 행동한다. 지독한 의심끝에 남...",1
4,"보고싶은 것만 보고, 본것을 믿으며 믿는만큼 생각하며 행동한다. 지독한 의심끝에 남...",1
5,"보고싶은 것만 보고, 본것을 믿으며 믿는만큼 생각하며 행동한다. 지독한 의심끝에 남...",1


In [117]:
data = pd.concat([df_augmentation_1, df_augmentation_0])

In [118]:
data

,content,label
0,전력삼국지 몇번다시보다가 다시 영화봤는데 이해가 빠르네요 관우랑 안량이랑 일기토할때...,1
1,전력삼국지 몇번다시보다가 다시 영화봤는데 이해가 빠르네요 관우랑 안량이랑 일기토할때...,1
2,전력삼국지 몇번다시보다가 다시 영화봤는데 이해가 빠르네요 관우랑 안량이랑 일기토할때...,1
3,"보고싶은 것만 보고, 본것을 믿으며 믿는만큼 생각하며 행동한다. 지독한 의심끝에 남...",1
4,"보고싶은 것만 보고, 본것을 믿으며 믿는만큼 생각하며 행동한다. 지독한 의심끝에 남...",1
...,...,...
1546,이영화에는 등급이없다 라는 영화보고 보게된 영화. 힐러리스웽크 다시보게됨,0
1547,이영화에는 등급이없다 라는 영화보고 보게된 영화. 힐러리스웽크 다시보게됨,0
1548,제 배종옥 김갑수 주연의 '세상에서 가장 아름다운 이별'이란 영화도 심금을 울립니다...,0
1549,배종옥 김갑수 주연의 '세상에서 가장 아름다운 이별'이란 영화도 심금을 울립니다. ...,0


In [119]:
data = data.sample(frac=1)

In [120]:
data['label'].value_counts()

1    1551
0    1551
Name: label, dtype: int64

In [121]:
data.to_csv("/content/drive/Shareddrives/실전기계_project/data/data_spo_aug_ri_train_1210.csv")